In [1]:
import cv2  # OpenCV library for image processing
import mediapipe as mp  # MediaPipe library for hand tracking
import pyautogui  # PyAutoGUI library for controlling the mouse cursor

class HandDetector:
    def __init__(self, static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5):
        # Initialise HandDetector object with parameters for hand tracking
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=static_image_mode, max_num_hands=max_num_hands,
                                          min_detection_confidence=min_detection_confidence,
                                          min_tracking_confidence=min_tracking_confidence)
        self.mp_drawing = mp.solutions.drawing_utils

    def detect_landmarks(self, frame):
        # Convert BGR frame to RGB for compatibility with MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Process the frame to detect hand landmarks
        results = self.hands.process(image_rgb)
        landmarks = []
        if results.multi_hand_landmarks:
            # Select the hand with the highest confidence score
            max_confidence_hand = max(results.multi_handedness, key=lambda x: x.classification[0].score)
            max_confidence_index = results.multi_handedness.index(max_confidence_hand)
            hand_landmarks = results.multi_hand_landmarks[max_confidence_index]
            landmarks.append(hand_landmarks.landmark)
            # Draw hand landmarks on the frame
            self.mp_drawing.draw_landmarks(frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
        return frame, landmarks

class CursorController:
    def __init__(self, screen_width, screen_height):
        # Initialise CursorController object with screen dimensions
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.is_fist = False
        self.is_middle_finger = False

    def move_cursor(self, x, y):
        # Move the cursor to the target position based on hand gestures
        target_x = int((1 - x) * self.screen_width)  # Invert x-axis as this was going the wrong way
        target_y = int(y * self.screen_height)
        pyautogui.moveTo(target_x, target_y)

    # Functions to simulate mouse actions
    def click(self):
        pyautogui.mouseDown()

    def release_click(self):
        pyautogui.mouseUp()

    def right_click(self):
        pyautogui.mouseDown(button='right')

    def release_right_click(self):
        pyautogui.mouseUp(button='right')

    def double_click(self):
        pyautogui.doubleClick()

def main():
    # Initialise video capture object
    cap = cv2.VideoCapture(0)
    # Get screen resolution
    screen_width = pyautogui.size().width
    screen_height = pyautogui.size().height
    # Initialise HandDetector and CursorController objects
    hand_detector = HandDetector()
    cursor_controller = CursorController(screen_width, screen_height)

    # Define distance variables outside the loop
    distance_thumb_index = 0
    distance_thumb_middle = 0

    while cap.isOpened():
        # Read frame from video capture
        ret, frame = cap.read()
        if not ret:
            break

        # Detect hand landmarks in the frame
        frame, landmarks = hand_detector.detect_landmarks(frame)

        # Add text instructions to the frame
        cv2.putText(frame, "PINCH INDEX TO THUMB TO LEFT CLICK", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, "MIDDLE TO THUMB TO RIGHT CLICK", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, "CROSS INDEX IN FRONT OF MIDDLE TO DOUBLE CLICK", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        if landmarks:
            thumb_tip = (landmarks[0][4].x, landmarks[0][4].y)
            index_tip = (landmarks[0][8].x, landmarks[0][8].y)
            middle_tip = (landmarks[0][12].x, landmarks[0][12].y)
            distance_thumb_index = ((thumb_tip[0] - index_tip[0])**2 + (thumb_tip[1] - index_tip[1])**2) ** 0.5
            distance_thumb_middle = ((thumb_tip[0] - middle_tip[0])**2 + (thumb_tip[1] - middle_tip[1])**2) ** 0.5
            
            # Double click detection: if index and middle fingers are close together
            distance_index_middle = ((index_tip[0] - middle_tip[0])**2 + (index_tip[1] - middle_tip[1])**2) ** 0.5
            # Threshold for double click
            double_click_threshold = 0.03

            # Left click action
            if distance_thumb_index < 0.05:
                if not cursor_controller.is_fist:
                    cursor_controller.click()
                    cursor_controller.is_fist = True
            else:
                if cursor_controller.is_fist:
                    cursor_controller.release_click()
                    cursor_controller.is_fist = False

            # Right click action
            if distance_thumb_middle < 0.05:
                if not cursor_controller.is_middle_finger:
                    cursor_controller.right_click()
                    cursor_controller.is_middle_finger = True
            else:
                if cursor_controller.is_middle_finger:
                    cursor_controller.release_right_click()
                    cursor_controller.is_middle_finger = False

            # Double click action
            if distance_index_middle < double_click_threshold:
                cursor_controller.double_click()

            palm_center = (landmarks[0][0].x, landmarks[0][0].y)  # Assuming the first landmark is the palm center.
            cursor_controller.move_cursor(palm_center[0], palm_center[1])

        # Display frame with hand landmarks
        cv2.imshow('Hand Tracking', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video capture object and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
